In [4]:
!pip -q install tiktoken duckduckgo-search

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import os



In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.191
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Custom Agents with Custom Search




In [6]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain.chains.llm import LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [7]:
# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [8]:
obj = search.run("How can I treat a sprained ankle?")
type(obj)

str

c:\Users\WORK\OneDrive\Desktop\drug_bot_agent\venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [9]:
obj

"Without proper treatment and rehabilitation, a severely injured ankle may not heal well and could lose its range of motion and stability, resulting in recurrent sprains and more downtime in the future. Anatomy of an ankle sprain. The most common type of ankle sprain is an inversion injury, or lateral ankle sprain. Sprains happen when a ligament is torn or damaged, usually when one of your joints moves further than it should. Muscle strains happen when one of your muscles is torn. People also sometimes call strains pulled muscles or muscle tears. Providers sometimes call tendon tears strains. Sprains and strains are both common sports injuries. Initial home treatment for a sprained ankle includes the RICE method (Rest, Ice, Compression, Elevation), over-the-counter pain relievers, gentle exercises, and using a brace or wrap. Applying a warm compress, gentle massage, and possibly physical therapy can aid recovery if the sprain is severe. Preventative measures include strengthening ... A

In [10]:
search.run("site:webmd.com How can I treat a sprained ankle?")

'Wound Care Step 3: Wash Around It. 8 /13. Put a little bit of soap on a washcloth and wash around the injured spot. Try not to get soap on the wound. Don\'t use hydrogen peroxide or iodine, because ... 11 /14. This stands for "rest," "ice," "compression," and "elevation.". It\'s most useful for sprains and broken bones, but it can help if your feet are swollen for other reasons ... 1 /19. Deep vein thrombosis is a blood clot that forms inside a vein, usually deep within your leg. As many as 900,000 Americans a year get one, and up to 100,000 die because of it. The danger is ... 8. Shed extra weight. Losing extra weight can provide a powerful energy boost, says Stewart, of Johns Hopkins University. Even small reductions in body fat improve mood, vigor, and quality of life. 2 /23. Ultra-high heels force the feet into a position that puts stress on the ball of the foot. At this joint, the long metatarsal bones meet the pea-shaped sesamoid bones and the toe bones ...'

c:\Users\WORK\OneDrive\Desktop\drug_bot_agent\venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [11]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [12]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [13]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [14]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Custom Output Parser
The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc

In [15]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [16]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [25]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(temperature=0,model='gemini-pro',google_api_key=os.getenv('GEMINI_KEY'))

## Define the stop sequence
This is important because it tells the LLM when to stop generation.

This depends heavily on the prompt and model you are using. Generally, you want this to be whatever token you use in the prompt to denote the start of an Observation (otherwise, the LLM may hallucinate an observation for you).

## Set up the Agent
We can now combine everything to set up our agent

In [26]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [27]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [28]:
tool_names

['Search WebMD']


### What is an Agent Executor?

Agent Executors take an agent and tools and use the agent to decide which tools to call and in what order.



In [29]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [32]:
agent_executor.run("How can I treat a spained ankle?")



> Entering new AgentExecutor chain...
Thought: I should search WebMD for information on treating a sprained ankle
Action: Search WebMD
Action Input: How to treat a sprained ankle

Observation:11 /14. This stands for "rest," "ice," "compression," and "elevation.". It's most useful for sprains and broken bones, but it can help if your feet are swollen for other reasons ... All high heels boost the risk of an ankle sprain.The most common, a lateral sprain, happens when you roll onto the outside of the foot. This stretches the ankle ligaments beyond their normal length. 1 /19. Deep vein thrombosis is a blood clot that forms inside a vein, usually deep within your leg. As many as 900,000 Americans a year get one, and up to 100,000 die because of it. The danger is ... Exercise also "promotes weight loss, which is helpful for joint pain and improves balance and coordination," Callahan said. Physical activity can also strengthen the muscles around the joints ... Gargle with salt water. It's 

'The RICE method is the most common treatment for a sprained ankle. RICE stands for rest, ice, compression, and elevation. Resting the ankle will help to reduce swelling and pain. Applying ice to the ankle will also help to reduce swelling and pain. Compressing the ankle with an elastic bandage will help to support the ankle and reduce swelling. Elevating the ankle will help to reduce swelling and pain.'

c:\Users\WORK\OneDrive\Desktop\drug_bot_agent\venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


### in Debug mode

In [31]:
langchain.debug = False

In [24]:
agent_executor.run("How can I treat a spained ankle?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I treat a spained ankle?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How can I treat a spained ankle?"
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:GoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\nSearch WebMD: useful for when you need to answer medical and pharmalogical questions\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search WebMD]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation

c:\Users\WORK\OneDrive\Desktop\drug_bot_agent\venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


[tool/end] [1:tool:duckduckgo_search] [2.70s] Exiting Tool run with output:
"11 /14. This stands for "rest," "ice," "compression," and "elevation.". It's most useful for sprains and broken bones, but it can help if your feet are swollen for other reasons ... 2 /23. Ultra-high heels force the feet into a position that puts stress on the ball of the foot. At this joint, the long metatarsal bones meet the pea-shaped sesamoid bones and the toe bones ... 1 /19. Deep vein thrombosis is a blood clot that forms inside a vein, usually deep within your leg. As many as 900,000 Americans a year get one, and up to 100,000 die because of it. The danger is ... Wound Care Step 3: Wash Around It. 8 /13. Put a little bit of soap on a washcloth and wash around the injured spot. Try not to get soap on the wound. Don't use hydrogen peroxide or iodine, because ... Gargle with salt water. It's a good way to soothe a throbbing throat. The salt water eases swelling and loosens mucus. Stir one-quarter to one-ha

'The RICE method is a common treatment for sprains. RICE stands for rest, ice, compression, and elevation. Resting the injured ankle will help to reduce swelling and pain. Applying ice to the ankle will also help to reduce swelling and pain. Compressing the ankle with an elastic bandage will help to reduce swelling. Elevating the ankle above the level of the heart will also help to reduce swelling.'

c:\Users\WORK\OneDrive\Desktop\drug_bot_agent\venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [ ]:
langchain.debug = False

## Adding a Conversation Memory
If you want to add memory to the agent, you’ll need to:

1. Add  **`chat_history`** into the custom prompt

2. Add the memory object to the agent executor when defining it.

In [ ]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [ ]:
agent_executor.run("How can I treat a sprained ankle?")



> Entering new AgentExecutor chain...

Thought: I need to find out what the best treatment for a sprained ankle is.
Action: Search WebMD
Action Input: "sprained ankle treatment"

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... Trauma to an area (jerking movements, auto accidents, falls, fractures, sprains, dislocations, and direct blows to the muscle) also can cause musculoskeletal pain. Other causes of pain include ... Being overwe

'To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. If the pain persists or worsens, it is important to seek medical attention.'

In [ ]:
agent_executor.run("What meds could I take?")



> Entering new AgentExecutor chain...


Thought: I need to know what medications are available to treat a sprained ankle.
Action: Search WebMD
Action Input: Sprained ankle medications

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... RICE. 2 /14. It stands for rest, ice, compress, and elevate, and it's the go-to treatment for minor sprains and bruising. Ice the injury as soon as possible and put pressure on it with an ... Trauma to an area (jerking movements, auto accidents,

 I now know the medications available to treat a sprained ankle.
Final Answer: To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. Other medications that may be used to treat a sprained ankle include over-the-counter pain relievers such as ibuprofen or acetaminophen, and topical creams or gels that contain menthol or capsaicin. If the pain persists or worsens, it is important to seek medical attention.

> Finished chain.


'To treat a sprained ankle, it is important to reduce swelling and pain in the first 24 hours after the injury. Apply a cold compress for 20-30 minutes, then remove it for 20-30 minutes. You can use a bag of frozen peas. Other medications that may be used to treat a sprained ankle include over-the-counter pain relievers such as ibuprofen or acetaminophen, and topical creams or gels that contain menthol or capsaicin. If the pain persists or worsens, it is important to seek medical attention.'

In [ ]:
agent_executor.run("How long will it take to heal?")



> Entering new AgentExecutor chain...

Thought: I need to find out how long it takes for a sprained ankle to heal.
Action: Search WebMD
Action Input: Sprained ankle healing time

Observation:3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or put ice ... Achilles Tendon Injury Symptoms. The most obvious sign is pain above your heel, especially when you stretch your ankle or stand on your toes. It may be mild and get better or worse over time. If ... Symptoms of muscle strain include: Swelling, bruising, or redness due to the injury. Pain at rest. Pain when the specific muscle or the joint in relation to that muscle is used. Weakness of the ... 7 /15. Over the next 4-21 days, you get a soft callus around the broken bone. This is when a substance called collagen moves in and slowly replaces the blood clot. The callus is stiffer than a ... Many Colors.

'It typically takes 4-21 days for a sprained ankle to heal. During this time, it is important to reduce swelling and pain by applying a cold compress for 20-30 minutes, then removing it for 20-30 minutes. If the pain persists or worsens, it is important to seek medical attention.'